#### Glob
主要查找符合特定规则的文件路径名。他用的匹配符比较简单，主要只用三个：$*， ？（单个）， []（范围）$<br>
- glob.glob返回所有匹配的文件路径列表。参数是parameter：glob.glob(r"E:\Picture\*\*.jpg")
- glob.iglob() 获取一个可遍历的对象，和glob.glob的区别在于，iglob返回的是一个iterator。

#### from __future__ import division
导入python未来支持的语言特征division(精确除法)，当我们没有在程序中导入该特征时，"/"操作符执行的是截断除法(Truncating Division),当我们导入精确除法之后，"/"执行的是精确除法

#### math.ceil(x)
return the ceiling of x as an Integral. This is the smallest integer >=x

In [4]:
from __future__ import division
from glob import glob
from six.moves import xrange

import os
import time
import math
import tensorflow as tf
import numpy as np

In [4]:
class DCGAN(object):
    """
    模型类，功能应该包括提供建模，训练， 评价接口
    """
    def __init__(self, sess, input_height=108, input_width=108, crop=True,
            batch_size=64, sample_num=64, output_height=64, output_width=64,
            y_dim=None, z_dim=100, gf_dim=64, df_dim=64, 
            gfc_dim=1024, dfc_dim=1024, c_dim=3, dataset_name='default',
            input_fname_pattern='*.jpg', checkpoint_dir=None, sample_dir=None):
        """
        init即初始化，即初始化模型的一些super params并加载或构建模型。
        参数一般包括：
        1. 因为层数是固定的所以无法作为参数，但每层的神经元数量，激活函数是可以做为参数的
        2. 为了能够断点续航，session是必须的。checkpoint_dir一般是当前路径。可以作为参数，同样的还有数据的路径，包括训练和测试的数据。
        3. 上面参数中不理解的是crop,sample_num,sample_dir, dataset_name参数，不知道是指什么参数
        """
        self.sess = sess
        self.crop = crop
        
        self.input_height = intput_height
        self.input_width = input_width
        self.output_height = output_height
        self.output_width = output_width
        
        self.batch_size = batch_size
        self.sample_num = sample_num
        
        self.y_dim = y_dim
        self.z_dim = z_dim
        
        self.gf_dim = gf_dim
        self.df_dim = df_dim
        
        self.gfc_dim = gfc_dim
        self.dfc_dim = dfc_dim
        
        # batch normalization: deals with poor initialization helps gradient flow
        self.d_bn1 = batch_norm(name='d_bn1')
        self.d_bn2 = batch_norm(name='d_bn2')
        
        # 不理解y_dim指的是什么，不存在y_dim就加一个bn也不知道是什么意思？？
        if not self.y_dim:
            self_d_bn3 = batch_norm(name='d_bn3')
        
        # 为什么这里又是从0开始了，这个逻辑也是不认的
        self.g_bn0 = batch_norm(name='g_bn0')
        self.g_bn1 = batch_norm(name='g_bn1')
        self.g_bn2 = batch_norm(name='g_bn2')
        
        if not self.y_dim:
            self_g_bn3 = batch_norm(name='g_bn3')
        
        self.dataset_name = dataset_name
        self.input_fname_pattern = input_fname_pattern
        self.checkpoint_dir = checkpoint_dir
        
        # 这里的X大写y小写， 究竟有什么意思呢？？
        if self.dataset_name == 'mnist':
            self.data_X, self.data_y = self.load_mnist()
            self.c_dim = self.data_X[0].shape[-1] # channel
        """
        else:
            self.data = glob(os.path.join("./data", self.dataset_name, self.input_fname_pattern))
            imreadImg = imread(self.data[0])
            if len(imreadImg.shape) >= 3:
                self.c_dim = imread(self.data[0].shape[-1]) #imreadImg.shape不一样吗？
            else:
                self.c_dim = 1
        self.grayscale = (self.c_dim == 1)
        """
        self.build_model()

    def discriminator(self, image, y=None, reuse=False):
        with tf.variable_scope("discriminator") as scope:
            if reuse: # 不知道是干嘛用的？
                scope.reuse_variables()
            if not self.y_dim:
                h0 = lrelu(conv2d(image, self.df_dim, name='d_h0_conv'))
                h1 = lrelu(self.d_bn1(conv2d(h0, self.df_dim*2, name='d_h1_conv')))
                h2 = lrelu(self.d_bn2(conv2d(h1, self.df_dim*4, name='d_h2_conv')))
                h3 = lrelu(self.d_bn3(conv2d(h2, self.df_dim*8, name='d_h2_conv')))
                h4 = linear(tf.reshape(h3, [self.batch_size, -1]), 1, 'd_h4_li') #为什么要分batch输入
                
                return tf.nn.sigmoid(h4), h4
            else:
                # 下面的不理解为什么要进行concat，他的好处是什么
                yb = tf.reshape(y, [self.batch_size, 1, 1, self.y_dim])
                x = conv_cond_concat(image, yb)
                
                h0 = lrelu(conv2d(x, self.c_dim + self.df_dim, name = 'd_h0_conv'))
                h0 = conv_cond_concat(h0, yb)
                
                h1 = lrelu(self.d_bn1(conv2d(h0, self.df_dim + self.y_dim, name='d_h1_conv')))
                h1 = tf.reshape(h1, [self.batch_size, -1])
                h1 = concat([h1, y1], 1)
                
                h2 = lrelu(self.d_bn2(linear(h1, self.dfc_dim, 'd_h2_lin')))
                h2 = concat([h2, y], 1)
                
                h3 = linear(h2, 1, 'd_h3_lin')
                
                return tf.nn.sigmoid(h3), h3
    
    def generator(self, z, y = None):
        with tf.variable_scope("generator") as scope:
            if not self.y_dim:
                s_h, s_w = self.output_height, self.output_width
                s_h2, s_w2 = conv_out_size_same(s_h, 2), conv_out_size_same(s_w, 2)
                s_h4, s_w4 = conv_out_size_same(s_h2, 2), conv_out_size_same(s_w2, 2)
                s_h8, s_w8 = conv_out_size_same(s_h4, 2), conv_out_size_same(s_w4, 2)
                s_h16, s_w16 = conv_out_size_same(s_h8, 2), conv_out_size_same(s_w8, 2)
                
                self.z_, self.h0_w, self.h0_b = linear(z, self.gf_dim*8*s_h16*s_w16, 'g_h0_lin', with_w=True)
                self.h0 = tf.reshape(self.z_, [-1, s_h16, s_w16, self.gf_dim*8])
                
                h0 = tf.nn.relu(self.g_bn0(self.h0))
                
                self.h1, self.h1_w, self.h1_b = deconv2d(h0, [self.batch_size, s_h8, s_w8, self.gf_dim*4],
                                                        name='g_h1', with_w=True)
                h1 = tf.nn.relu(self.g_bn1(self.h1))
                
                self.h2, self.h2_w, self.h2_b = deconv2d(h1, [self.batch_size, s_h4, s_w4, self.gf_dim*2],
                                                        name='g_h2', with_w=True)
                h2 = tf.nn.relu(self.g_bn2(self.h2))
                
                self.h3, self.h3_w, self.h3_b = deconv2d(h2, [self.batch_size, s_h2, s_w2, self.gf_dim],
                                                        name='g_h3', with_w=True)
                h3 = tf.nn.relu(self.g_bn3(self.h3))
                
                h4, self.h4_w, self.h4_b = deconv2d(h3, [self.batch_size, s_h, s_w, self.c_dim],
                                                   name='g_h4', with_w=True)
                return tf.nn.tanh(h4)
            else:
                s_h, s_w = self.output_height, self.output_width
                s_h2, s_w2 = int(s_h/2), int(s_w/2) # 为什么不向上取整了？？？
                s_h4, s_w4 = int(s_h/4), int(s_w/4)
                
                yb = tf.reshape(y, [self.batch_size, 1, 1, self.y_dim])
                z = concat([z, y], 1) #不是扩展channel维了吗？？
                
                h0 = tf.nn.relu(self.g_bn0(linear(z, self.gfc_dim, 'g_h0_lin')))
                h0 = concat([h0, y], 1)
                
                h1 = tf.nn.relu(self.g_bn1(linear(h0, self.gf_dim*2*s_h4*s_w4, )))
                h1 = tf.reshape(h1, [self.batch_size, s_h4, s_w4, self.gf_dim*2])
                h1 = conv_cond_concat(h1, yb)
                
                h2 = tf.nn.relu(self.g_bn2(deconv2d(h1, [self.batch_size, s_h2, s_w2, self.gf_dim*2], name='g_h2')))
                h2 = conv_cond_concat(h2, yb)
                
                return tf.nn.sigmoid(deconv2d(h2, [self.batch_size, s_h, s_w, self.c_dim], name='g_h3'))
                
                
        
    def conv_out_size_same(size, stride):
        return int(math.ceil(float(size)/float(stride)))
    
    def load_mnist(self):
        """
        读mnist，这里是按unit8（字节）读，而且前几位是图片信息，这里没有做图片信息分析，是因为图片信息已知。
        mnist集前16字节是图片信息，前4为验证码，随后是图片数目，cols， hors
        """
        data_dir = os.path.join("./data", self.dataset_name)
        
        # 没关文件
        fd = open(os.path.join(data_dir, 'train-images-idx3-ubyte'))
        loaded = np.fromfile(file=fd, dtype=np.unit8) #都是unit8吗
        trX = loaded[16:].reshape((-1, 28, 28, 1)).astype(np.float) #16是怎么来的
        
        fd = open(os.path.join(data_dir, 'train-labels-idx1-ubyte'))
        loaded = np.fromfile(file=fd, dtype=np.unit8)
        trY = loaded[8:].reshape((-1)).astype(np.float) #这个reshape有什么意义吗？？一维转一维？？
        
        fd = open(os.path.join(data_dir, 'test-images-idx3-ubyte'))
        loaded = np.fromfile(file=fd, dtype=np.unit8)
        teX = loaded[16:].reshape((-1, 28, 28, 1)).astype(np.float)
        
        fd = open(os.path.join(data_dir, 'test-labels-idx1-ubyte'))
        loaded = np.fromfile(file=fd, dtpye=np.unit8)
        teY = loaded[8:].reshape((-1)).astype(np.float)
        
        # 必须吗，那X呢
        trY = np.asarray(trY)
        teY = np.asarray(teY)
        
        # 把数据都连一起了，不测试了？？
        X = np.concatenae((trX, teX), axis = 0)
        y = np.concatenate((trY, teY), axis = 0).astype(np.int)
        
        # 醉了，还能这样shuffle pair，也是够了
        seed = 547
        np.random.seed(seed)
        np.random.shuffle(X)
        np.random.seed(seed)
        np.random.shuffle(y)
        
        y_vec = np.zeros((len(y), self.y_dim), dtype=np.float) #len(y)行，y_dim为标签的维度
        
        # 猜测：因为目标是数值识别，所以这里y_dim可能是9，然后，下面这步就是one hot了
        for i, label in enumerate(y):
            y_vec[i, y[i]] = 1.0 #可以用label的
        
        #/255是为啥？？？算normalization？
        return X/255., y_vec

#### np.random.seed(seed=None)
This method is called when `RandomState` is initialized. It can be
called again to re-seed the generator.

#### sigmoid_cross_entropy_with_logits(_sentinel=None, labels=None, logits=None, name=None)
Computes sigmoid cross entropy given logits.<br>
Measures the probability error in discrete classification tasks in which each class is independent and not mutually exclusive. For instance, one could perform multilabel classification where a picture can contain both an elephant and a dog at the same time.

In [23]:
?np.zeros()

#### tf.concat(concat_di, values, name)
tf.concat(concat_dim, values, name='concat')<br>
concat_dim是tensor连接的方向（维度），values是要连接的tensor链表，name是操作名。cancat_dim维度可以不一样，其他维度的尺寸必须一样。下面举两个例子：两个二维tensor连接，两个三维tensor连接。<br>
##### 两个二维tensor连接：<br>
concat_dim：0表示行，1表示列<br>
t1 = [[1,2,3], [4,5,6]]<br>
t2 = [[7,8,9], [10,11,12]]<br>
tf.concat(0, [t1, t2]) ==> [[1,2,3], [4,5,6], [7,8,9], [10,11,12]]<br>
tf.concat(1, [t1, t2]) ==> [[1,2,3,7,8, 9], [4,5,6,10,11, 12]]<br>

In [5]:
import math
import numpy as np
import tensorflow as tf

from tensorflow.python.framework import ops

# 考虑到版本问题， 做一下协调
try:
    image_summary = tf.image_summary
    scalar_summary = tf.scalar_summary
    histogram_summary = tf.histogram_summary
    merge_summary = tf.merge_summary
    SummaryWriter = tf.train.SummaryWriter
except:
    image_summary = tf.summary.image
    scalar_summary = tf.summary.scalar
    histogram_summary = tf.summary.histogram
    merge_summary = tf.summary.merge
    SummaryWriter = tf.summary.FileWriter

if "concat_v2" in dir(tf):
    def concat(tensors, axis, *args, **kwargs):
        return concat_v2(tensors, axis, *args, **kwargs)
else:
    def concat(tensors, axis, *args, **kwargs):
        return concat(tensors, axis, *args, **kwargs)

# 建batch_norm类

### 建batch_norm类
batch_norm就是batch normalization<br>
tf.contrib.layers.batch_norm(input, decay=0.9, updates_collections=None, epsilon=1e-5, scale=True, is_training=True, scope="bn")<br>
- inputs: A tensor with 2 or more dimensions, where the first dimension has batch_size. The normalization is over all but the last dimension if data_format is NHWC and the second dimension if data_format is NCHW.<br>
- decay: Decay for the moving average. Reasonable values for decay are close to 1.0, typically in the multiple-nines range: 0.999, 0.99, 0.9, etc. Lower decay value (recommend trying decay=0.9) if model experiences reasonably good training performance but poor validation and/or test performance. Try zero_debias_moving_mean=True for improved stability.<br>
- updates_collections: Collections to collect the update ops for computation. The updates_ops need to be executed with the train_op. If None, a control dependency would be added to make sure the updates are computed in place.<br>
- epsilon: Small float added to variance to avoid dividing by zero.<br>
- scope: Optional scope for variable_scope<br>
- is_training: Whether or not the layer is in training mode. In training mode it would accumulate the statistics of the moments into moving_mean and moving_variance using an exponential moving average with the given decay. When it is not in training mode then it would use the values of the moving_mean and the moving_variance.<br>
- scope: Optional scope for variable_scope<br>

该方法返回的是一个 tensor，使用示例如下：<br>
x = tf.placeholder(tf.float32, [64, 28,28,1])<br>
w= tf.truncated_normal([5,5,1,32], stddev=0.1)<br>
b = tf.constant(0.1, shape=[32])<br>
h = tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding='SAME') + b<br>
h_bn = tf.contrib.layers.batch_norm(h, decay=0.9, updates_collections=None, epsilon=1e-5, scale=True, is_training=True, scope="bn")<br>
h_r = tf.nn.relu(h_bn)<br>
主要解释来自知乎问题“深度学习中 Batch Normalization为什么效果好”<br>
1. What is BN?
顾名思义，batch normalization嘛，就是“批规范化”咯。Google在ICML文中描述的非常清晰，即在每次SGD时，通过mini-batch来对相应的activation做规范化操作，使得结果（输出信号各个维度）的均值为0，方差为1. 而最后的“scale and shift”操作则是为了让因训练所需而“刻意”加入的BN能够有可能还原最初的输入（即当），从而保证整个network的capacity。（有关capacity的解释：实际上BN可以看作是在原模型上加入的“新操作”，这个新操作很大可能会改变某层原来的输入。当然也可能不改变，不改变的时候就是“还原原来输入”。如此一来，既可以改变同时也可以保持原输入，那么模型的容纳能力（capacity）就提升了。）<br>
关于DNN中的normalization，大家都知道白化（whitening），只是在模型训练过程中进行白化操作会带来过高的计算代价和运算时间。因此本文提出两种简化方式：1）直接对输入信号的每个维度做规范化（“normalize each scalar feature independently”）；2）在每个mini-batch中计算得到mini-batch mean和variance来替代整体训练集的mean和variance. 这便是Algorithm 1.
3. Where to use BN?
BN可以应用于网络中任意的activation set。文中还特别指出在CNN中，BN应作用在非线性映射前,即对x=Wu+b做规范化。
4. Why BN?
好了，现在才是重头戏－－为什么要用BN？BN work的原因是什么？那BN到底是什么原理呢？说到底还是为了防止“梯度弥散”。
5. When to use BN?
OK，说完BN的优势，自然可以知道什么时候用BN比较好。例如，在神经网络训练时遇到收敛速度很慢，或梯度爆炸等无法训练的状况时可以尝试BN来解决。另外，在一般使用情况下也可以加入BN来加快训练速度，提高模型精度。

####  __call__方法
关于__call__方法，不得不先提到一个概念，就是可调用对象callable，我们平时自定义函数、内置函数和类都属于可调用对象，但凡是可以把一堆括号（）应用到某个对象身上都可称为可调用对象判断对象是否为可调用对象可以用函数callable()<br>
如果在类中实现了__call__方法，那么实例对象也将称为一个可调用对象，实例对象也可以像函数一样作为可调用对象来用。<br>
那么什么场景用得上呢？这个要结合类的特性来说，类可以记录数据（属性），而函数不行（闭包某种意义上也可以），利用这种特性可以实现基于类的装饰器，在类里面记录状态。

In [8]:
class batch_norm(object):
    def __init__(self, epsilon=1e-5, momentum=0.9, name="batch_norm"):
        """
        不是很理解，为什么要加这个
        """
        with tf.variable_scope(name):
            self.epsilon = epsilon
            self.momentum = momentum
            self.name = name
    
    def __call__(self, x, train=True):
        return tf.contrib.layers.batch_norm(x, 
                                           decay = self.momentum,
                                           updates_collections=None,
                                           epsilon=self.epsilon,
                                           scale=True,
                                           is_training=True,
                                           scope=self.name)

In [9]:
def conv_cond_concat(x,y):
    """
    x.shape: [self.batch_size, image_cols, image_hor, channel]
    y.shape: [self.batch_size, 1, 1, self.y_dim]
    y*tf.ones[]的结果，形状上来讲核x是差不多的，不过x中图片上不同位置上的值是不同的，然而在这里则都是应该根据y值来决定，对应目标的
    channel（还是当成channel来想象，比较好理解）上，全是1，而被其他的都是0.
    concat所以结果就成了[self.batch_size, image_cols, image_hor, channel+self.y+dim]。也就是说，channel 变多了。
    前几个原图片的channel，后几个为新加入的。
    """
    x_shapes = x.get_shape()
    y_shapes = y.get_shape()
    return concat([x, y*tf.ones([x_shapes[0], x_shapes[1], x_shapes[2], y_shapes[3]])], 3)

#### 记几个方法
##### get_variable(name, shape=None, dtype=None, initializer=None, regularizer=None, trainable=True, collections=None, caching_device=None, partitioner=None, validate_shape=True, use_resource=None, custom_getter=None)
Gets an existing variable with these parameters or create a new one.

##### tf.truncated_normal_initializer
Initializer that generates a truncated normal distribution.<br>
These values are similar to values from a random_normal_initializer except that values more than two standard deviations from the mean are discarded and re-drawn. This is the recommended initializer for neural network weights and filters.<br>
__init__(<br>
    mean=0.0,<br>
    stddev=1.0,<br>
    seed=None,<br>
    dtype=tf.float32<br>
)

##### tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None, data_format=None, name=None)
Computes a 2-D convolution given 4-D input and filter tensors.<br>
Given an input tensor of shape [batch, in_height, in_width, in_channels] and a filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]

##### tf.nn.bias_add(value, bias, data_format=None, name=None)
Adds bias to value.<br>
This is (mostly) a special case of tf.add where bias is restricted to 1-D. Broadcasting is supported, so value may have any number of dimensions. Unlike tf.add, the type of bias is allowed to differ from value in the case where both types are quantized.

In [10]:
def conv2d(input_, output_dim, k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02, name="conv2d"):
    """filter: [height, width, output_channels, input_channels]"""
    with tf.variable_scope(name):
        w = tf.get_variable('w', [k_h, k_w, input_.get_shape()[-1], output_dim],
                            initializer = tf.truncated_normal_initializer(stddev=stddev))
        conv = tf.nn.conv2d(input_, w, strides=[1, d_h, d_w, 1], padding='SAME')
        biases = tf.get_variable('biases', [output_dim], initializer=tf.constant_initializer(0.0))
        conv = tf.reshape(tf.nn.bias_add(conv, biases), conv.get_shape())
        return conv

#### 继续几个方法
##### tf.nn.conv2d_transpose(value, filter, output_shape, strides, padding='SAME', data_format='NHWC', name=None)
The transpose of conv2d: This operation is sometimes called "deconvolution" after Deconvolutional Networks, but is actually the transpose (gradient) of conv2d rather than an actual deconvolution.

##### tf.nn.conv2d
是旧的版本（0.7.0以前），现在看不到了


In [12]:
def deconv2d(input_, output_shape, k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02, name="deconv2d", with_w=False):
    with tf.variable_scope(name):
        w = tf.get_variable('w', [k_h, k_w, output_shape[-1], input_.get_shape()[-1]],
                            initializer=tf.random_normal_initializer(stddev=stddev))
        try:
            deconv = tf.nn.conv2d_transpose(input_, w, output_shape=output_shape, strides=[1, d_h, d_w, 1])
        except:
            deconv = tf.nn.deconv2d(input_, w, output_shape=output_shape,
                                    strides=[1, d_h, d_w, 1])
        biases = tf.get_variable('biases', [output_shape[-1]], initializer=tf.constant_initializer(0.0))
        deconv = tf.reshape(tf.nn.bias_add(deconv, biases), deconv.get_shape())
        if with_w:
            return deconv, w, biases
        else:
            return deconv

In [2]:
def lrelu(x, leak=0.2, name="lrelu"):
    """leaky relu: 大于零保持，小于零打折"""
    return max(x, leak*x)

In [1]:
def linear(input_, output_size, scope=None, stddev=0.02, bias_start=0.0, with_w=False):
    shape = input_.get_shape().as_list() #****不该相乘吗？？
    with tf.variable_scope(scope or "Linear"):
        matrix = tf.get_variable("Matrix", [shape[1], output_size], tf.float32,
                                 initializer = tf.random_normal_initializer(stddev=stddev))
        bias = tf.get_variable("bias", [output_size],
                              initializer=tf.constant_initializer(bias_start))
        if with_w:
            return tf.matmul(input_, matrix) + bias, matrix, bias
        else:
            return tf.matmul(input_, matrix) + bias

In [2]:
import tensorflow as tf
import numpy as np

x = tf.placeholder(tf.float32, [1, 1])
y = tf.placeholder(tf.float32, [1, 1])
h0 = li(x, 10, 'in')
h0 = tf.nn.sigmoid(h0)
tf.contrib.layers.batch_norm(h0, 
                           decay = 0.9,
                           updates_collections=None,
                           epsilon=1e-5,
                           scale=True,
                           is_training=True)
out = li(h0, 1, 'out')
squared_deltas = tf.square(out - y)
loss = tf.reduce_sum(squared_deltas)
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [5]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
X = list(np.random.normal(0, 10, 10000))
noise = list(np.random.normal(0, 1, 10000))
counter = 1

for label, i in enumerate(X):
    a = np.matrix([i])
    b = np.matrix([i*i+ noise[label]])
    sess.run(train, {x: a, y: b})
    counter = counter + 1
    if counter%100000 == 0:
        print(a)
        print(sess.run(h0, {x: a}))
        print(sess.run(out, {x: a}))
        print(sess.run(loss, {x: a, y: b}))

sess.close()


In [1]:
def li(input_, output_size, scope=None, stddev=0.02, bias_start=0.0, with_w=False):
    shape = input_.shape  #****不该相乘吗？？
    with tf.variable_scope(scope or "Linear"):
        matrix = tf.get_variable("Matrix", [shape[1], output_size], tf.float32,
                                 initializer = tf.random_normal_initializer(stddev=stddev))
        bias = tf.get_variable("bias", [output_size],
                              initializer=tf.constant_initializer(bias_start))
        return tf.matmul(input_, matrix) + bias

In [2]:
def lrelu(x, leak = 0.2):
    return max(x, tf.matmul([[leak]], x))

In [4]:
import tensorflow as tf

matrix = li2()
print(matrix)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(matrix))

Tensor("MatMul:0", shape=(1, 10), dtype=float32)
[[-0.04569849 -0.03433954 -0.03440236 -0.03189389  0.00359198  0.01259415
   0.01644778 -0.00417289  0.00840337  0.02925177]]


In [9]:
X = list(np.random.normal(0, 10, 10000))
noise = list(np.random.normal(0, 1, 10000))

for i, j in .(X, noise):
    print(i, j)

ValueError: too many values to unpack (expected 2)

In [7]:
X = np.random.normal(0, 10, 10000)


array([  6.93290543, -14.95265141,   4.46539758, ...,  -9.89825101,
        20.13026969,  10.51131503])

In [4]:
print(max(1,2))

2
